In [3]:
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma

In [8]:
from langchain.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

/var/folders/42/94lj0cp519ncj7_m79j3_h3m0000gn/T/ipykernel_4581/1822540165.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
/opt/anaconda3/envs/ollama-rag/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
from langchain.schema import Document

# create Langchain document for IPL players

doc1 = Document(
        page_content="Virat Kohli is one of the most successful and consistent batsmen in IPL history. Known for his aggressive batting style and fitness, he has led the Royal Challengers Bangalore in multiple seasons.",
        metadata={"team": "Royal Challengers Bangalore"}
    )
doc2 = Document(
        page_content="Rohit Sharma is the most successful captain in IPL history, leading Mumbai Indians to five titles. He's known for his calm demeanor and ability to play big innings under pressure.",
        metadata={"team": "Mumbai Indians"}
    )
doc3 = Document(
        page_content="MS Dhoni, famously known as Captain Cool, has led Chennai Super Kings to multiple IPL titles. His finishing skills, wicketkeeping, and leadership are legendary.",
        metadata={"team": "Chennai Super Kings"}
    )
doc4 = Document(
        page_content="Jasprit Bumrah is considered one of the best fast bowlers in T20 cricket. Playing for Mumbai Indians, he is known for his yorkers and death-over expertise.",
        metadata={"team": "Mumbai Indians"}
    )
doc5 = Document(
        page_content="Ravindra Jadeja is a dynamic all-rounder who contributes with both bat and ball. Representing Chennai Super Kings, his quick fielding and match-winning performances make him a key player.",
        metadata={"team": "Chennai Super Kings"}
    )

In [5]:
docs = [doc1 , doc2 , doc3 , doc4 , doc5]

In [9]:
vector_store = Chroma(
    embedding_function=embeddings , 
    persist_directory='chroma_db' , 
    collection_name='sample'
)

In [10]:
# add documents
vector_store.add_documents(docs)

['8bcd19cd-ed5d-4b3f-9664-46b98c11e46f',
 '5dc3861a-a289-4d34-ba46-754d0b582604',
 'cdbde09e-d86a-4a18-a545-208c8e99503f',
 'e856fdbb-97f5-49ec-a5aa-7136f30c273a',
 '4ab49171-07ab-4dd4-87c1-9287b7c55f7c']

In [12]:
# view documents
vector_store.get(include=['embeddings' , 'documents' , 'metadatas'])

{'ids': ['8bcd19cd-ed5d-4b3f-9664-46b98c11e46f',
  '5dc3861a-a289-4d34-ba46-754d0b582604',
  'cdbde09e-d86a-4a18-a545-208c8e99503f',
  'e856fdbb-97f5-49ec-a5aa-7136f30c273a',
  '4ab49171-07ab-4dd4-87c1-9287b7c55f7c'],
 'embeddings': array([[ 0.00994719,  0.06914335, -0.05147116, ..., -0.03543337,
          0.01284809,  0.01248283],
        [ 0.00127741,  0.03129852, -0.02375373, ..., -0.00518364,
         -0.03280607,  0.02737714],
        [-0.10265923,  0.02650806,  0.02271505, ..., -0.0335975 ,
         -0.07984944, -0.01507706],
        [ 0.02123401, -0.02468547, -0.0449437 , ..., -0.10995813,
          0.00572558,  0.09915373],
        [ 0.01873972,  0.04382848, -0.0430425 , ..., -0.07801614,
         -0.07840677, -0.00304191]]),
 'documents': ['Virat Kohli is one of the most successful and consistent batsmen in IPL history. Known for his aggressive batting style and fitness, he has led the Royal Challengers Bangalore in multiple seasons.',
  "Rohit Sharma is the most successful ca

In [16]:
# search documents
vector_store.similarity_search(
    query='Who is among these a bowler?' , 
    k = 2
)

[Document(id='e856fdbb-97f5-49ec-a5aa-7136f30c273a', metadata={'team': 'Mumbai Indians'}, page_content='Jasprit Bumrah is considered one of the best fast bowlers in T20 cricket. Playing for Mumbai Indians, he is known for his yorkers and death-over expertise.'),
 Document(id='5dc3861a-a289-4d34-ba46-754d0b582604', metadata={'team': 'Mumbai Indians'}, page_content="Rohit Sharma is the most successful captain in IPL history, leading Mumbai Indians to five titles. He's known for his calm demeanor and ability to play big innings under pressure.")]

In [17]:
# search with similarity score
vector_store.similarity_search_with_score(
    query='Who among these are a bowler?' , 
    k = 2
)

[(Document(id='e856fdbb-97f5-49ec-a5aa-7136f30c273a', metadata={'team': 'Mumbai Indians'}, page_content='Jasprit Bumrah is considered one of the best fast bowlers in T20 cricket. Playing for Mumbai Indians, he is known for his yorkers and death-over expertise.'),
  0.9693600817970106),
 (Document(id='5dc3861a-a289-4d34-ba46-754d0b582604', metadata={'team': 'Mumbai Indians'}, page_content="Rohit Sharma is the most successful captain in IPL history, leading Mumbai Indians to five titles. He's known for his calm demeanor and ability to play big innings under pressure."),
  1.1493451444109168)]

In [18]:
# meta-data filtering
vector_store.similarity_search_with_score(
    query="" , 
    filter={"team":"Chennai Super Kings"}
)

[(Document(id='cdbde09e-d86a-4a18-a545-208c8e99503f', metadata={'team': 'Chennai Super Kings'}, page_content='MS Dhoni, famously known as Captain Cool, has led Chennai Super Kings to multiple IPL titles. His finishing skills, wicketkeeping, and leadership are legendary.'),
  1.843600615215233),
 (Document(id='4ab49171-07ab-4dd4-87c1-9287b7c55f7c', metadata={'team': 'Chennai Super Kings'}, page_content='Ravindra Jadeja is a dynamic all-rounder who contributes with both bat and ball. Representing Chennai Super Kings, his quick fielding and match-winning performances make him a key player.'),
  1.8909372836429765)]

In [19]:
# update documents
updated_doc1 = Document(
    page_content="Virat Kohli, the former captain of Royal Challengers Bangalore (RCB), is renowned for his aggressive leadership and consistent batting performances. He holds the record for the most runs in IPL history, including multiple centuries in a single season. Despite RCB not winning an IPL title under his captaincy, Kohli's passion and fitness set a benchmark for the league. His ability to chase targets and anchor innings has made him one of the most dependable players in T20 cricket.",
    metadata={"team": "Royal Challengers Bangalore"}
)

vector_store.update_document(document_id='09a39dc6-3ba6-4ea7-927e-fdda591da5e4', document=updated_doc1)

Update of nonexisting embedding ID: 09a39dc6-3ba6-4ea7-927e-fdda591da5e4
Update of nonexisting embedding ID: 09a39dc6-3ba6-4ea7-927e-fdda591da5e4


In [20]:
# view documents
vector_store.get(include=['embeddings','documents', 'metadatas'])

{'ids': ['8bcd19cd-ed5d-4b3f-9664-46b98c11e46f',
  '5dc3861a-a289-4d34-ba46-754d0b582604',
  'cdbde09e-d86a-4a18-a545-208c8e99503f',
  'e856fdbb-97f5-49ec-a5aa-7136f30c273a',
  '4ab49171-07ab-4dd4-87c1-9287b7c55f7c'],
 'embeddings': array([[ 0.00994719,  0.06914335, -0.05147116, ..., -0.03543337,
          0.01284809,  0.01248283],
        [ 0.00127741,  0.03129852, -0.02375373, ..., -0.00518364,
         -0.03280607,  0.02737714],
        [-0.10265923,  0.02650806,  0.02271505, ..., -0.0335975 ,
         -0.07984944, -0.01507706],
        [ 0.02123401, -0.02468547, -0.0449437 , ..., -0.10995813,
          0.00572558,  0.09915373],
        [ 0.01873972,  0.04382848, -0.0430425 , ..., -0.07801614,
         -0.07840677, -0.00304191]]),
 'documents': ['Virat Kohli is one of the most successful and consistent batsmen in IPL history. Known for his aggressive batting style and fitness, he has led the Royal Challengers Bangalore in multiple seasons.',
  "Rohit Sharma is the most successful ca

In [21]:
# delete document
vector_store.delete(ids=['09a39dc6-3ba6-4ea7-927e-fdda591da5e4'])

Delete of nonexisting embedding ID: 09a39dc6-3ba6-4ea7-927e-fdda591da5e4
Delete of nonexisting embedding ID: 09a39dc6-3ba6-4ea7-927e-fdda591da5e4


In [22]:
# view documents
vector_store.get(include=['embeddings','documents', 'metadatas'])

{'ids': ['8bcd19cd-ed5d-4b3f-9664-46b98c11e46f',
  '5dc3861a-a289-4d34-ba46-754d0b582604',
  'cdbde09e-d86a-4a18-a545-208c8e99503f',
  'e856fdbb-97f5-49ec-a5aa-7136f30c273a',
  '4ab49171-07ab-4dd4-87c1-9287b7c55f7c'],
 'embeddings': array([[ 0.00994719,  0.06914335, -0.05147116, ..., -0.03543337,
          0.01284809,  0.01248283],
        [ 0.00127741,  0.03129852, -0.02375373, ..., -0.00518364,
         -0.03280607,  0.02737714],
        [-0.10265923,  0.02650806,  0.02271505, ..., -0.0335975 ,
         -0.07984944, -0.01507706],
        [ 0.02123401, -0.02468547, -0.0449437 , ..., -0.10995813,
          0.00572558,  0.09915373],
        [ 0.01873972,  0.04382848, -0.0430425 , ..., -0.07801614,
         -0.07840677, -0.00304191]]),
 'documents': ['Virat Kohli is one of the most successful and consistent batsmen in IPL history. Known for his aggressive batting style and fitness, he has led the Royal Challengers Bangalore in multiple seasons.',
  "Rohit Sharma is the most successful ca